#İl isimlerinin bulunduğu txt dosyasını okumak ve veri alacağımız linke uygun hale getirmek.

#Örnek link: https://www.yemeksepeti.com/city/adana

#Not sadece il merkezli arama yapıyoruz. İlçe verilerini almıyoruz. İsteyen onu da alabilir.

In [9]:
from unidecode import unidecode

with open("iller.txt","r",encoding="utf-8") as dosya:
    iller=dosya.readlines()
iller=["/city/"+unidecode(i.strip().lower()) for i in iller]
print(iller)

['/city/adana', '/city/adiyaman', '/city/afyonkarahisar', '/city/agri', '/city/aksaray', '/city/amasya', '/city/ankara', '/city/antalya', '/city/ardahan', '/city/artvin', '/city/aydin', '/city/balikesir', '/city/bartin', '/city/batman', '/city/bayburt', '/city/bilecik', '/city/bingol', '/city/bitlis', '/city/bolu', '/city/burdur', '/city/bursa', '/city/canakkale', '/city/cankiri', '/city/corum', '/city/denizli', '/city/diyarbakir', '/city/duzce', '/city/edirne', '/city/elazig', '/city/erzincan', '/city/erzurum', '/city/eskisehir', '/city/gaziantep', '/city/giresun', '/city/gumushane', '/city/hakkari', '/city/hatay', '/city/igdir', '/city/isparta', '/city/istanbul', '/city/izmir', '/city/kahramanmaras', '/city/karabuk', '/city/karaman', '/city/kars', '/city/kastamonu', '/city/kayseri', '/city/kilis', '/city/kirikkale', '/city/kirklareli', '/city/kirsehir', '/city/kocaeli', '/city/konya', '/city/kutahya', '/city/malatya', '/city/manisa', '/city/mardin', '/city/mersin', '/city/mugla', '/c

#Tüm illere ait linkleri toplamak

In [10]:
urls=[]
for i in iller:
    url=f"https://www.yemeksepeti.com{i}"
    urls.append(url)
print(urls)

['https://www.yemeksepeti.com/city/adana', 'https://www.yemeksepeti.com/city/adiyaman', 'https://www.yemeksepeti.com/city/afyonkarahisar', 'https://www.yemeksepeti.com/city/agri', 'https://www.yemeksepeti.com/city/aksaray', 'https://www.yemeksepeti.com/city/amasya', 'https://www.yemeksepeti.com/city/ankara', 'https://www.yemeksepeti.com/city/antalya', 'https://www.yemeksepeti.com/city/ardahan', 'https://www.yemeksepeti.com/city/artvin', 'https://www.yemeksepeti.com/city/aydin', 'https://www.yemeksepeti.com/city/balikesir', 'https://www.yemeksepeti.com/city/bartin', 'https://www.yemeksepeti.com/city/batman', 'https://www.yemeksepeti.com/city/bayburt', 'https://www.yemeksepeti.com/city/bilecik', 'https://www.yemeksepeti.com/city/bingol', 'https://www.yemeksepeti.com/city/bitlis', 'https://www.yemeksepeti.com/city/bolu', 'https://www.yemeksepeti.com/city/burdur', 'https://www.yemeksepeti.com/city/bursa', 'https://www.yemeksepeti.com/city/canakkale', 'https://www.yemeksepeti.com/city/canki

#Toplanan linklere tek tek istek atmak ve o ilde olan firma isimlerini sonuc adlı listede toplamak

#Eğer hata alırsanız muhtemelen site aşırı istekten sizi engellemiştir. 5 dk kadar bekleyip tekrar deneyin.

In [5]:
import cloudscraper
from bs4 import BeautifulSoup

def firmaadları(link):
    scraper=cloudscraper.CloudScraper()
    req=scraper.get(link)
    req.encoding="utf-8"
    soup=BeautifulSoup(req.text,"html.parser")
    etiketler=soup.find_all("div",{"class":"bds-c-grid-item vendor webfound-refresh"})

    sonuc=[]
    for etiket in etiketler:
        firmaisim=etiket.find("div", {"class": "vendor-name f-title-small-font-size fw-title-small-font-weight lh-title-small-line-height ff-title-small-font-family ffs-title-small-font-feature-settings"})
        if firmaisim:
            firmaisim=firmaisim.get_text(strip=True)

            link=etiket.find("a")
            href=link.get('href')

            sonuc.append({"Firma":firmaisim,"Href": href})
    if sonuc:
        df={item["Firma"]: item["Href"] for item in sonuc}
        return df
    return {}


#Firma isimlerini firma kodlarını ve hangi ilde olduklarını bir araya toplamak için boş dataframe açıyoruz.

In [11]:
import pandas as pd

veri=pd.DataFrame(columns=["Firma Adı","Firma Kodu","İl"])

for url in urls:
    firmalar=firmaadları(url) 
    il=url.split("/")[-1]
    for firma_adı,firma_href in firmalar.items():
        firma_kodu=firma_href.split("/")[-2]
        veri=veri._append({"Firma Adı":firma_adı,"Firma Kodu":firma_kodu,"İl":il},ignore_index=True)
        
veri["İl"]=veri["İl"].str.capitalize()

print(veri)

                                     Firma Adı Firma Kodu         İl
0     Meşhur Adıyaman Çiğ Köftecisi Ömer Aybak       fdjc      Adana
1                                Mr. Çiğ Köfte       pmlc      Adana
2                                         Köse       u2bz      Adana
3                            Bolulu Hasan Usta       cr4o      Adana
4                             Starbucks Coffee       ues6      Adana
...                                        ...        ...        ...
3575                               Helvacı Ali       ie2s  Zonguldak
3576                               Usta Pideci       a515  Zonguldak
3577                                Yummy Yumm       k1ce  Zonguldak
3578                               Hayal Döner       timb  Zonguldak
3579                          Çiğ Köfteci Sait       x0fw  Zonguldak

[3580 rows x 3 columns]


#Yorumlar için firma kodlarına ihtiyacımız var. O yüzden bir öncekinde onu hallettik

#Örnek link: https://reviews-api-tr.fd-api.com/reviews/vendor/tn40?global_entity_id=YS_TR&limit=100&created_at=desc&has_dish=true

#Linkteki "tn40" firma kodudur. Geriye kalanlar ise yorumların yeniden eskiye sıralanmasıdır

#Not bu linkler yorumlarda en iyi puan, düşükten yükseğe veya eskiden yeniye olarak değişebilir. Biz yanlı olmaması adına eskiden yeniye şeklinde aldık 

In [7]:
def yorumlar(firma_kodu):
    url=f"https://reviews-api-tr.fd-api.com/reviews/vendor/{firma_kodu}?global_entity_id=YS_TR&limit=100&created_at=desc&has_dish=true"
    scraper=cloudscraper.CloudScraper()
    req=scraper.get(url).json()["data"]

    yorum=[item["text"] for item in req]
    return yorum

#Yorumlar isimli fonksiyona bir önce elde ettiğimiz tüm firma kodlarını gönderdik ve yorumları aldık

#Bu yorumları veri isimli data setimizde Yorumlar isimli sütuna taşıdık ve daha sonra firma kodlarını sildik

#En sonda da "yorumlar" isminde csv olarak kaydettik

In [8]:
veri["Yorumlar"]=veri["Firma Kodu"].apply(lambda x: yorumlar(x))
veri.drop(columns=["Firma Kodu"],inplace=True)
veri.to_csv("yorumlar.csv",index=False,encoding="utf-8")